In [ ]:
import xmltodict
import json
import boto3
import json
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
# Load XML data from a file or string
with open("./xml_data.xml") as xml_file:
    data = xmltodict.parse(xml_file.read())
 
# Convert to JSON
# json_input = 
json_input = json.loads(json.dumps(data, indent=4))
 
# You can now work with the JSON data
print(json_input)

In [ ]:
import xmltodict
import json

def remove_nulls(d):
    """
    Recursively remove dictionary entries with None values.
    """
    if isinstance(d, dict):
        return {
            k: remove_nulls(v) 
            for k, v in d.items() 
            if v is not None
        }
    elif isinstance(d, list):
        return [remove_nulls(v) for v in d if v is not None]
    else:
        return d

def convert_xml_file_to_json(file_path):
    """
    Read XML from a file, convert it to JSON string, removing keys with null values.
    """
    with open(file_path, 'r') as file:
        xml_string = file.read()

    # Parse the XML string to a Python dictionary
    xml_dict = xmltodict.parse(xml_string)

    # Remove None values from dictionary
    cleaned_dict = remove_nulls(xml_dict)

    # Convert the cleaned dictionary to a JSON string
    json_string = json.dumps(cleaned_dict, indent=4)

    return json_string



In [ ]:
# Example Usage
file_path = './xml_data.xml'
json_result = convert_xml_file_to_json(file_path)
print(json_result)

In [ ]:
def chunk_by_lines(data, lines_per_chunk):
    lines = data.split('\n')
    total_chunks = len(lines) // lines_per_chunk + (1 if len(lines) % lines_per_chunk > 0 else 0)
    chunklist=[]
    for i in range(total_chunks):
        print(i)
        chunk_lines = lines[i * lines_per_chunk:(i + 1) * lines_per_chunk]
        chunk_data = '\n'.join(chunk_lines)
        chunklist.append(chunk_data)
    return chunklist

In [ ]:
chunklist = chunk_by_lines(json_result, 25)

In [ ]:
bedrock_runtime = boto3.client(
        aws_access_key_id= os.getenv('ACCESS_KEY'),
        aws_secret_access_key=os.getenv('SECRET_ACCESS_KEY'),
        service_name='bedrock-runtime', 
        region_name='us-east-1'
    )

def use_llm_with_aws_bedrock(prompt):
    payload = json.dumps({ 
        'prompt': prompt,
        'max_gen_len': 2000,
        'top_p': 0.9,
        'temperature': 0.2
    })

    response = bedrock_runtime.invoke_model(
        body=payload, 
        modelId="meta.llama2-70b-chat-v1", 
#         modelId = "meta-textgeneration-llama-2-7b-f",
        accept='*/*', 
        contentType='application/json'
    )

    body = response.get('body').read().decode('utf-8')
    response_body = json.loads(body)
    # print(response_body['generation'].strip())

    return response_body['generation'].strip()
    

In [ ]:
def construct_prompt(chunk):
    _prompt = f"""
        Human:Translate the values within json key value pair from various languages to English, ensuring the json structure remains unchanged. Only the values inside the json should be translated, not the keys themselves. 
        The result should only contain the json data.
        
        DO NOT ADD ANY  EXTRA TAGS. THAT IS NOT AT ALL TOLERABLE.
        
        data: {chunk}
        Assistant:"""
    return _prompt

In [ ]:
_prompt = f"""
    Human:Given a split of JSON object, read each key-value pair. For each value, detect the language, translate it into English if it's not already in English, and provide the response in the following format:
    DO NOT ADD ANY  EXTRA KEYS. THAT IS NOT AT ALL TOLERABLE.
    MAKE SURE THE JSON FORMAT IS NOT CHANGED.
 
 data::{{
                    "reactionuniversallyuniqueid": "f13ace24-7576-327f-92e9-f0ac6ae1dc0e",
                    "primarysrcreactreportedlang": "nld",
                    "primarysrcreactinnativelang": "Reactie op of rond de injectieplaats: pijn",}}

Assistant:"""

In [ ]:
prompt = f"""Given a split of JSON object, read each key-value pair. For each value, detect the language, translate it into English if it's not already in English.
 
JSON Object:{{
            "reactionuniversallyuniqueid": "f13ace24-7576-327f-92e9-f0ac6ae1dc0e",
            "primarysrcreactreportedlang": "nld",
            "primarysrcreactinnativelang": "Reactie op of rond de injectieplaats: pijn",}}
 
Expected Response Format:
- tag: greeting
  original_text: Hola
  detected_language: Spanish
  translated_text: Hello
 
Please process the JSON object and provide the responses in the specified format."""

In [ ]:
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
B_INST, E_INST = "[INST]", "[/INST]"
sys_msg = "<s>" + B_SYS + """Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Given a split of JSON object, read each key-value pair. For each value, detect the language, translate it into English if it's not already in English, and provide the response in the following format:
 
- tag: [key name]
- original_text: [original value]
- detected_language: [detected language of the value]
- translated_text: [translated value into English]
    ```

Here are some previous conversations between the Assistant and User:

User: JSON Object:
{{
    "greeting": "Hola"
}}
Assistant: ```json
{{ "tag": "greeting",
 "original_text": "Hola"
  "detected_language": "Spanish"
  "translated_text": "Hello"}}
```
User: {{
    "farewell": "Au revoir"
}}
Assistant: ```json
{{"tag": "farewell"
  "original_text": "Au revoir"
  "detected_language": "French"
  "translated_text": "Goodbye"}}
```
User: {{
    "question": "Wie geht es Ihnen?"
}}
Assistant: ```json
{{"tag": "question"
  "original_text": "Wie geht es Ihnen?"
  "detected_language": "German"
  "translated_text": "How are you?"}}
```


Here is the latest conversation between Assistant and User.""" + E_SYS

instruction = sys_msg + B_INST + " Respond to the following in JSON with original_text,  detected_language and translated_text" + E_INST
human_msg =  instruction + """\nUser: { "reactionuniversallyuniqueid": "f13ace24-7576-327f-92e9-f0ac6ae1dc0e",
            "primarysrcreactreportedlang": "nld",
            "primarysrcreactinnativelang": "Reactie op of rond de injectieplaats: pijn",}"""

In [ ]:
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
B_INST, E_INST = "[INST]", "[/INST]"
sys_msg = "<s>" + B_SYS + """Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Given a split of JSON object, read each key-value pair. For each value, detect the language, translate it into English if it's not already in English, and provide the response in the following format:
 
- tag: [key name]
- original_text: [original value]
- detected_language: [detected language of the value]
- translated_text: [translated value into English]
    ```

Here are some previous conversations between the Assistant and User:

User: JSON Object:
{{
    "greeting": "Hola"
}}
Assistant: ```json
{{ "tag": "greeting",
 "original_text": "Hola"
  "detected_language": "Spanish"
  "translated_text": "Hello"}}
```



Here is the latest conversation between Assistant and User.""" + E_SYS

instruction = sys_msg + B_INST + " Respond to the following in JSON with original_text,  detected_language and translated_text" + E_INST
human_msg =  instruction + """\nUser: { "reactionuniversallyuniqueid": "f13ace24-7576-327f-92e9-f0ac6ae1dc0e",
            "primarysrcreactreportedlang": "nld",
            "primarysrcreactinnativelang": "Reactie op of rond de injectieplaats: pijn",}"""

In [ ]:
# prompt = construct_prompt(chunklist[1])
result = use_llm_with_aws_bedrock(human_msg)

In [ ]:
print(result)

In [ ]:
{
            "safetyreportid": "BL-LRB-23462",
            "transmissiondater3": "20221015000000",
            "reporttype": "1",
            "receivedater3": "20221015000000+0200",
            "receiptdater3": "20221015000000+0200",
            "additionaldocumentr3": "false",
            "fulfillexpeditecriteriar3": null,
            "companynumb": "NL-LRB-00839381",
            "casesendertype": "1",
            "duplicater3": "[NI]",
            "casenullification": null,
            "nullificationreason": null,
            "primarysource": {
                "reportertitle": null,
                "reportergivename": null,
                "reportermiddlename": null,
                "reporterfamilyname": null,
                "reporterorganization": null,
                "reporterdepartment": null,
                "reporterstreet": null,
                "reportercity": null,
                "reporterstate": null,
                "reporterpostcode": null,
                "reporterphone": null,
                "reportercountryr3": "NL",
                "qualificationr3": "5",
                "primarysourceregpurpose": "1"
            }

In [ ]:
input = {
                    "reactionuniversallyuniqueid": "939db665-f91c-3f8c-9764-ef544640fa7d",
                    "primarysrcreactreportedlang": "nld",
                    "primarysrcreactinnativelang": "Niet lekker voelen",
                    "reactionmeddraversionllt": "25.0",
                    "reactionmeddrallt": "10025482",
                    "reactionoutcome": "3",
                    "medicallyconfirm": "false"
                }

In [ ]:
llm_response = [{ "tag": "reactionuniversallyuniqueid",
                 "original_text": "939db665-f91c-3f8c-9764-ef544640fa7d",
                 "detected_language": "English",
  "translated_text": "939db665-f91c-3f8c-9764-ef544640fa7d"},
 { "tag": "primarysrcreactreportedlang",
 "original_text": "nld",
  "detected_language": "English",
  "translated_text": "nld"},
 { "tag": "primarysrcreactinnativelang",
 "original_text": "Niet lekker voelen",
  "detected_language": "Spanish",
  "translated_text": "Dummy translated text"},
 { "tag": "reactionmeddraversionllt",
 "original_text": "25.0",
  "detected_language": "English",
  "translated_text": "25.0"},
 { "tag": "reactionmeddrallt",
 "original_text": "10025482",
  "detected_language": "English",
  "translated_text": "10025482"},
 { "tag": "reactionoutcome",
 "original_text": "3",
  "detected_language": "English",
  "translated_text": "3"},
 { "tag": "medicallyconfirm",
 "original_text": "false",
  "detected_language": "English",
  "translated_text": "false"}]

In [ ]:
def update_json_values(json1, json2):
    # Iterate through each item in json2
    for item in json2:
        # Check if the detected language is not English
        if item['detected_language'] != 'English':
            print(item)
            # Extract the tag, original text, and translated text
            tag = item['tag']
            original_text = item['original_text']
            translated_text = item['translated_text']

            # Define a function to recursively search and replace values in json1
            def update_recursive(d, key, old_value, new_value):
#                 print(type(d))
                
                if isinstance(d, dict):
                    for k, v in d.items():
                        if isinstance(v, (dict, list)):
                            update_recursive(v, key, old_value, new_value)
                        elif k == key and v == old_value:
                            d[k] = new_value

                elif isinstance(d, list):
                    for item in d:
                        if isinstance(item, (dict, list)):
                            update_recursive(item, key, old_value, new_value)

            # Update json1 with the new value
            update_recursive(json1, tag, original_text, translated_text)

    return json1

json1 = json_input

json2 = llm_response
# Example usage
updated_json = update_json_values(json1, json2)


In [ ]:
print(type(updated_json))

In [ ]:
import xml.etree.ElementTree as ET

def dict_to_xml(input_dict, root_element):
    """
    Convert a nested dictionary into an XML string.
    """
    def build_element(parent, dict_data):
        if isinstance(dict_data, dict):
            for key, val in dict_data.items():
                sub_element = ET.SubElement(parent, key)
                build_element(sub_element, val)
        elif isinstance(dict_data, list):
            for item in dict_data:
                item_element = ET.SubElement(parent, parent.tag)
                build_element(item_element, item)
        else:
            parent.text = str(dict_data) if dict_data is not None else ''

    root = ET.Element(root_element)
    build_element(root, input_dict)
    return ET.tostring(root, encoding='unicode')

def save_xml(input_dict, root_element, file_name):
    """
    Save a dictionary as an XML file.
    """
    xml_data = dict_to_xml(input_dict, root_element)
    with open(file_name, 'w') as file:
        file.write(xml_data)

# Example usage
save_xml(updated_json, 'icsreport', './output.xml')
